## Tema 5: Proyección y recorte 3D

In [1]:
from OpenGL.GL import *
from OpenGL.GLUT import *
from OpenGL.GLU import *

import igv3 as igv

In [2]:
from math import sqrt
from math import cos
from math import sin
from math import tan
from math import pi

### Función de inicialización de la librería con los valores deseados

In [3]:
def init_gl():
    glutInit()                                     # Inicializa la librería GLUT
    glutInitDisplayMode(GLUT_SINGLE | GLUT_RGB | GLUT_DEPTH)    # Único frame buffer y modo de color RGBA
    glutInitWindowSize(600, 600)                   # (width, height)
    glutInitWindowPosition(100, 100)               # (x pos, y pos)
    glutCreateWindow(b'PROYECCION 3D')             # Creación de la ventana (si no se pone b da error)
    glClearColor(1.0, 1.0, 1.0, 1.0);              # Color del buffer

    glEnable(GL_DEPTH_TEST)                        # HABILITA COMPROBACIÓN DE PROFUNDIDAD EN EL DIBUJO           

### Función que dibuja una casa

La función `house` dibuja una casa formada por 6 polígonos. La denominación de los polígonos (cara frontal, cara posterior, etc) asume el punto de vista por defecto de OpenGL (punto de vista en (0, 0, 0) y punto de referencia en (0, 0, -1)).


In [4]:
def house():
    
    # Definición de vértices
    v1 = [0.0, 0.0, 1.0]
    v2 = [1.0, 0.0, 1.0]
    v3 = [1.0, 1.0, 1.0]
    v4 = [0.0, 1.0, 1.0]
    v5 = [0.0, 0.0, -1.0]
    v6 = [1.0, 0.0, -1.0]
    v7 = [1.0, 1.0, -1.0]
    v8 = [0.0, 1.0, -1.0]
    v9 = [0.5, 1.5, 1.0]
    v10 = [0.5, 1.5, -1.0]
    
    # Definición de colores
    white = [1.0, 1.0, 1.0]
    black = [0.0, 0.0, 0.0]
    light_grey = [240/255, 240/255, 244/255]
    dark_grey = [125/255, 125/255, 125/255]
    red = [204/255, 22/255, 52/255]
        
    # CARA FRONTAL
    glColor3fv(light_grey)
    glBegin(GL_POLYGON)
    glVertex3fv(v1)
    glVertex3fv(v2)
    glVertex3fv(v3)
    glVertex3fv(v9)
    glVertex3fv(v4)
    glEnd()
   
    # CARA DERECHA
    glColor3fv(light_grey)
    glBegin(GL_POLYGON)
    glVertex3fv(v2)
    glVertex3fv(v6)
    glVertex3fv(v7)
    glVertex3fv(v3)
    glEnd()
  
    # CARA IZQUIERDA
    glColor3fv(dark_grey)
    glBegin(GL_POLYGON)
    glVertex3fv(v1)
    glVertex3fv(v5)
    glVertex3fv(v8)
    glVertex3fv(v4)
    glEnd()
   
    # CARA POSTERIOR
    glColor3fv(dark_grey)
    glBegin(GL_POLYGON)
    glVertex3fv(v5)
    glVertex3fv(v6)
    glVertex3fv(v7)
    glVertex3fv(v10)
    glVertex3fv(v8)
    glEnd()
    
    # AGUA DERECHA TEJADO
    glColor3fv(red)
    glBegin(GL_POLYGON)
    glVertex3fv(v3)
    glVertex3fv(v7)
    glVertex3fv(v10)
    glVertex3fv(v9)
    glEnd()
    
    # AGUA IZQUIERDA TEJADO
    glColor3fv(red)
    glBegin(GL_POLYGON)
    glVertex3fv(v4)
    glVertex3fv(v9)
    glVertex3fv(v10)
    glVertex3fv(v8)
    glEnd()
  
    # BORDES
    glColor3fv(black)
    glLineWidth(2)
    glBegin(GL_LINES)
    
    # cara frontal
    glVertex3fv(v1); glVertex3fv(v2)
    glVertex3fv(v2); glVertex3fv(v3)
    glVertex3fv(v3); glVertex3fv(v9)
    glVertex3fv(v9); glVertex3fv(v4)
    glVertex3fv(v4); glVertex3fv(v1)
    # cara posterior
    glVertex3fv(v5); glVertex3fv(v6)
    glVertex3fv(v6); glVertex3fv(v7)
    glVertex3fv(v7); glVertex3fv(v10)
    glVertex3fv(v10); glVertex3fv(v8)
    glVertex3fv(v8); glVertex3fv(v5)    
    # lateral derecho
    glVertex3fv(v3); glVertex3fv(v7)
    glVertex3fv(v2); glVertex3fv(v6)
    # lateral izquierdo
    glVertex3fv(v4); glVertex3fv(v8)
    glVertex3fv(v1); glVertex3fv(v5)
    # tejado
    glVertex3fv(v9); glVertex3fv(v10)

    glEnd()
    

### Funciones de los viewports

La callback de display `display` muestra 6 viewports con distintas proyecciones del mundo, para ello invoca a las 6 funciones siguientes:

-  `viewport_cabinet`: muestra la proyección caballera del mundo
-  `viewport_ortho_front`: muestra la proyección ortogonal del mundo con el punto de vista por defecto (hacia Zworld-)
-  `viewport_ortho_rear`: muestra la proyección ortogonal del mundo con el punto de vista hacia Zworld+
-  `viewport_ortogonal_right`: muestra la proyección ortogonal del mundo con el punto de vista hacia XWorld-
-  `viewport_ortogonal_left`: muestra la proyección ortogonal del mundo con el punto de vista hacia XWorld+
-  `viewport_ortogonal_left`: muestra la proyección ortogonal del mundo con el punto de vista hacia Yworld-

In [5]:
def viewport_cabinet(xMin, xMax, yMin, yMax, dNear, dFar):

    # Definición del viewport
    glViewport(0,300,300,300)   # glViewport(x_lower_left_corner, y_lower_left_corner, width, height)

    
    ###############################################################
    # PREPARACIÓN DE LA PROYECCIÓN - SE ESTUDIARÁ POSTERIORMENTE
    ###############################################################
    
    # PREPARACIÓN DE LA MATRIZ DE CONVERSIÓN GABINETE
    factor = pi/180                 # Factor de conversión de grados a radianes
    alpha = 63.4                    # Definición del ángulo alfa
    alpha = alpha * factor          # Conversión a radianes
    phi = 30                        # Definición del ángulo phi 
    phi = phi * factor              # Conversión a radianes
    cx = cos(phi)/tan(alpha)
    cy = sin(phi)/tan(alpha)
    cabinet_matrix = [1, 0, 0, 0, 0, 1, 0, 0, cx, cy, 1, 0, 0, 0, 0, 1]
    
    # DEFINICIÓN DE LA MATRIZ DE PROYECCIÓN (tipo de proyección) 
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    glMultMatrixf(cabinet_matrix)   # definición de la proyección (tipo gabinete)
    
    ###############################################################
    
    # Definición del volumen de recorte
    glOrtho(xMin, xMax, yMin, yMax, dNear, dFar) 

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    # Definición de la posición de la cámara (punto de vista)
    # valor por defecto de lookAt (punto de vista hacia Zworld-)
    x0 = 0.0;  y0 = 0.0;  z0 = 0.0;  xref = 0.0;  yref = 0.0;  zref = -1.0;  vx = 0.0;  vy = 1.0; vz = 0.0
    gluLookAt(x0, y0, z0, xref, yref, zref, vx, vy, vz)   # Se podría eliminar porque se usa el valor por defecto
    
    # Dibujo de los ejes y la casa
    igv.axes(xMin, xMax, yMin, yMax, dNear, dFar, True)
    house()
    
    # Dibujo de una etiqueta
    #glRasterPos3fv([0.0, -1.0, 0.0])
    #label_text = bytes("GABINETE", encoding = "utf-8")
    #glutBitmapString(GLUT_BITMAP_8_BY_13, label_text)
    
    igv.draw_text_3d("GABINETE", 0.0, -1.0, 0.0)
    

In [6]:
def viewport_ortho_front(xMin, xMax, yMin, yMax, dNear, dFar):
    
    # Definición del viewport
    #glViewport(300, 300, 300,300)   # glViewport(x_lower_left_corner, y_lower_left_corner, width, height)
    glViewport(300,0,300,300)   # glViewport(x_lower_left_corner, y_lower_left_corner, width, height)
    
    # Preparación de la proyección ortogonal
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    
    glOrtho(xMin, xMax, yMin, yMax, dNear, dFar) 

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    # Definición de la posición de la cámara (punto de vista). Valor por defecto de lookAt (punto de vista hacia Zworld-)
    x0 = 0.0;  y0 = 0.0;  z0 = 0.0;  xref = 0.0;  yref = 0.0;  zref = -1.0;  vx = 0.0;  vy = 1.0; vz = 0.0
    gluLookAt(x0, y0, z0, xref, yref, zref, vx, vy, vz)   # Se podría eliminar porque se usa el valor por defecto
    
    # Dibujo de los ejes y la casa
    igv.axes(xMin, xMax, yMin, yMax, dNear, dFar, True)
    house()
    
    # Dibujo de una etiqueta  
    igv.draw_text_3d("VISTA FRONTAL", -1.0, -1.0, 0.0)

In [7]:
def viewport_ortho_rear(xMin, xMax, yMin, yMax, dNear, dFar):
    
    # Definición del viewport
    #glViewport(300,0,300,300)   # glViewport(x_lower_left_corner, y_lower_left_corner, width, height)
    glViewport(300, 300, 300,300)   # glViewport(x_lower_left_corner, y_lower_left_corner, width, height)
    
    # Preparación de la proyección ortogonal
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    
    glOrtho(xMin, xMax, yMin, yMax, dNear, dFar) 

    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    # Definición de la posición de la cámara (punto de vista). Punto de vista hacia Zworld+
    x0 = 0.0;  y0 = 0.0;  z0 = 0.0;  xref = 0.0;  yref = 0.0;  zref = 1.0;  vx = 0.0;  vy = 1.0; vz = 0.0
    gluLookAt(x0, y0, z0, xref, yref, zref, vx, vy, vz)
    
    # Dibujo de los ejes y la casa
    igv.axes(xMin, xMax, yMin, yMax, dNear, dFar, True)
    house()
    
    # Dibujo de una etiqueta
    igv.draw_text_3d("VISTA POSTERIOR", 1.0, -1.0, 0.0)
    

In [8]:
def viewport_ortho_right(xMin, xMax, yMin, yMax, dNear, dFar):
    
    # Definición del viewport
    #glViewport(600,300,300,300)   # glViewport(x_lower_left_corner, y_lower_left_corner, width, height)
    glViewport(0,0,300,300)   # glViewport(x_lower_left_corner, y_lower_left_corner, width, height)
    
    # Preparación de la proyección ortogonal
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    
    glOrtho(xMin, xMax, yMin, yMax, dNear, dFar) 
  
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()
    
    # Definición de la posición de la cámara (punto de vista). Punto de vista hacia Xworld-
    x0 = 0.0;  y0 = 0.0;  z0 =0.0;  xref = -1;  yref = 0.0;  zref = 0.0;  vx = 0.0;  vy = 1.0; vz = 0.0
    gluLookAt(x0, y0, z0, xref, yref, zref, vx, vy, vz)
    
    # Dibujo de los ejes y la casa
    igv.axes(xMin, xMax, yMin, yMax, dNear, dFar, True)
    house()
    
    # Dibujo de una etiqueta
    igv.draw_text_3d("VISTA LATERAL DERECHO", 0.0, -1.0, 2.0)
    

### Callback de display

In [9]:
def display():
 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)    # RESETEO DE LOS BUFFERS
    
    xMin = -3; xMax = 3; yMin = -3; yMax = 3
    
    dNear = -3;  dFar = 3  
      
    viewport_cabinet(xMin, xMax, yMin, yMax, dNear, dFar)
    viewport_ortho_front(xMin, xMax, yMin, yMax, dNear, dFar)
    viewport_ortho_rear(xMin, xMax, yMin, yMax, dNear, dFar)
    viewport_ortho_right(xMin, xMax, yMin, yMax, dNear, dFar)
    
    glFlush()
    

### Función principal que llama a las funciones de inicialización y de dibujo

In [10]:
def main():
    init_gl()
    glutDisplayFunc(display)
    glutMainLoop()    # Deja la ventana abierta a la espera de eventos
    

### Llamada a la función principal

In [ ]:
main()